# Get corrections at a corner reflector (CR) location using GridGeococoding

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

import numpy as np
from scipy.interpolate import interp2d
from matplotlib import pyplot as plt

import s1etad
from s1etad import Sentinel1Etad, ECorrectionType
from s1etad.geometry import GridGeocoding

## Searching bursts in which the CR is present

Load the S1-ETAD product.

In [ ]:
filename = '../../sample-products/S1B_IW_ETA__AXDH_20200127T113414_20200127T113858_020002_025D72_0096.SAFE'
eta = Sentinel1Etad(filename)

The CR position has been chosed to be in the overlab region of different bursts and swaths.

In [ ]:
from shapely.geometry import Point

lat0 = +70.13  # 70° 7'48.65"N
lon0 = -76.19  # 76°11'41.55"W
h0 = 366

cr = Point(lon0, lat0)

Query for burst cobering the CR.

In [ ]:
selection = eta.query_burst(geometry=cr)
selection

## Get corrections at CR location

Reatrieve the first burst.

In [ ]:
burst = next(eta.iter_bursts(selection))

Get tha grid of geodetic coordinates.

In [ ]:
lats, lons, heights = burst.get_lat_lon_height()

Get the range and azimuth time axes.

In [ ]:
azimuth_time, range_time = burst.get_burst_grid()

Initialize the Grid Geocoding object.

**NOTE**: if one uses time axis then consistent time coordinates shall be provided in all back-geocoding requests.

In [ ]:
ebg = GridGeocoding(lats, lons, heights, xaxis=range_time, yaxis=azimuth_time)

Now it is possible to perform the back-geocoding i.e. computation of RADAR coordinates (tau, t) starting form geodatic coordinates (lat, lon, h):

   (lat, lon, h) -> (tau, t) 

In [ ]:
tau, t = ebg.backward_geocode(lat0, lon0, h0)
tau, t

Of course it is also possible to make the inverse conversion:

In [ ]:
lat1, lon1, h1 = ebg.forward_geocode(tau, t)
print(f'Initial coordinates:      (lat0, lon0, h0) = ({lat0}, {lon0}, {h0})')
print(f'Foeward geocoding output: (lat1, lon1, h1) = ({lat1.item()}, {lon1.item()}, {h1.item()})')

### Using umage coordinates (linse, samples)

It is also possible to initialize the GridGeocoding without providing time axes information.

In this case the geocoder will work using image coorinates (lines and samples) instead of range/azimuth times.

In [ ]:
ebg = GridGeocoding(lats, lons, heights)

It is possible to perform back-geocoding, i.e. (lat, lon, h) -> (sample, line):

In [ ]:
sample, line = ebg.backward_geocode(lat0, lon0, h0)
sample, line

and also to perform the forward conversion: (sample, line) -> (lat, lon, h)

In [ ]:
lat1, lon1, h1 = ebg.forward_geocode(sample, line)
print(f'Initial coordinates:      (lat0, lon0, h0) = ({lat0}, {lon0}, {h0})')
print(f'Foeward geocoding output: (lat1, lon1, h1) = ({lat1.item()}, {lon1.item()}, {h1.item()})')

## Putting all together

In [ ]:
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(nrows=len(selection), ncols=1, figsize=[13, 8])

for loop, burst in enumerate(eta.iter_bursts(selection)):
    # coordinate grids
    lats, lons, heights = burst.get_lat_lon_height()

    # backward geocoding with image coordinates
    ebg = GridGeocoding(lats, lons, heights)
    x0, y0 = ebg.backward_geocode(lat0, lon0, h0) 
    print("xy  ", burst.swath_id, burst.burst_index, x0, y0,
          ebg.latitude(x0, y0), ebg.longitude(x0, y0), ebg.height(x0, y0))

    # get the range and azimuth times
    azimuth_time, range_time = burst.get_burst_grid()

    # backward geocoding with time coordinates
    ebg = GridGeocoding(lats, lons, heights,
                        xaxis=range_time, yaxis=azimuth_time)
    tau0, t0 = ebg.backward_geocode(lat0, lon0, h0) 
    print("time", burst.swath_id, burst.burst_index, tau0, t0,
          ebg.latitude(tau0, t0), ebg.longitude(tau0, t0), ebg.height(tau0, t0))
    
    # correction
    cor = burst.get_correction(s1etad.ECorrectionType.SUM, meter='True')

    ax[loop].imshow(cor['x'], aspect='auto')
    rec_half_size = 1
    p = Rectangle((x0 - rec_half_size, y0 - rec_half_size),
                  width=rec_half_size*2+1, height=rec_half_size*2+1,
                  color='red', fill=True)
    ax[loop].add_patch(p)

    # interpolate at the desired working (RADAR) coordinates
    f_t  = interp2d(range_time, azimuth_time, cor['x'])

    # get the image (lines and samples) axes
    yaxis = np.arange(azimuth_time.size)
    xaxis = np.arange(range_time.size)
    
    # interpolate at the desired working (image) coordinates
    f_ij = interp2d(xaxis, yaxis, cor['x'])
    
    print(f"Interpoaltion by array coordinate {f_ij(x0, y0)} or time {f_t(tau0,t0)} should be the same")
    print(f"The total correction at lat/lon {lat0, lon0} is  {f_ij(x0, y0)} m in range")
    print()

In [ ]:
burst.radar_to_geodetic(tau0, t0) 

In [ ]:
cor